In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import rioxarray
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import datacube

<hr style="border:2px solid gray"> </hr>

# Create dataset

In [ ]:
# Create array for wave height
height = xr.DataArray(np.array([[1,2,3],[4,5,6]]), dims=("latitude", "longitude"), coords={"latitude": [1,2], "longitude": [1,2,3]})
height.attrs['long_name'] = 'Wave height'
height = height.astype('int16', copy=False)
height

In [ ]:
# Create array for wave period
period = height.copy(deep=True, data=np.array([[7,8,9],[10,11,12]]))
period.attrs['long_name'] = 'Wave period'  
period = period.astype('int16', copy=False)
period

In [ ]:
height.plot();

In [ ]:
# Create dataset with wave height and period
ds = xr.Dataset({"height": height, "period": period})
ds.rio.write_crs(4326, inplace=True)
#ds.rio.write_transform(inplace=True)
ds

<hr style="border:2px solid gray"> </hr>

# Save dataset to disk

In [ ]:
# Save dataset to netCDF file
file = "minimal-example.nc"
ds.to_netcdf(path=file, mode="w")

In [ ]:
# Check file with rio/gdalinfo
#!rio info $file | jq
#!gdalinfo $file

<hr style="border:2px solid gray"> </hr>

# Load dataset from disk

In [ ]:
# Load dataset from disk
ds_disk = xr.open_dataset(file)
ds_disk

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=(15,5))
ds_disk.height.plot(ax=axes[0]);
ds_disk.period.plot(ax=axes[1]);
plt.tight_layout()

<hr style="border:2px solid gray"> </hr>

# Load dataset from Open Data Cube

In [ ]:
# Create datacube instance
dc = datacube.Datacube()

In [ ]:
# List available products
product_list = dc.list_products()
product_list

In [ ]:
# List available datasets/measurements
measurement_list = dc.list_measurements()
measurement_list

In [ ]:
# Define parameters
product = "minimal_example_eo3"
output_crs = "EPSG:4326"
resolution = (1, 1)
align = (0.5, 0.5)
measurements = ["height", "period"]

#query = {"align": align}
query = {"latitude": (1, 2), "longitude": (1, 3), "align": align, "measurements": measurements}

In [ ]:
# output_crs and resolution are required for eo3 metadata
ds_datacube = dc.load(product, 
                      output_crs=output_crs, 
                      resolution=resolution, 
                      **query)
ds_datacube

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=(15,5))
ds_datacube.height.plot(ax=axes[0]);
ds_datacube.period.plot(ax=axes[1]);
plt.tight_layout()